In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/My\ Drive/kaggle.json ~/.kaggle/

# **Project 1-** predicting IMBD reviews

In [ ]:
# Importing essential libraries and functions

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from numpy import array

from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
%cd /content/drive/My Drive/project deep
! ls

/content/drive/My Drive/project deep
'Copy of a1_IMDB_Dataset.csv'	'Copy of a3_IMDb_Unseen_Reviews.csv'
'Copy of a2_glove.6B.100d.txt'	'Copy of c1_lstm_model_acc_0.864.h5'


In [ ]:
df = pd.read_csv("Copy of a1_IMDB_Dataset.csv")
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [ ]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [ ]:
df['review'][3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [ ]:
def preprocess_text(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stepwords composed of A-Z & a-z only
    in lowercase'''

    sentence = sen.lower()

    # Remove html tags
    sentence = re.sub(r'<[^>]+>',' ',sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  # When we remove apostrophe from the word "Mark's", the apostrophe is replaced by an empty space. Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  # Next, we remove all the single characters and replace it by a space which creates multiple spaces in our text. Finally, we remove the multiple spaces from our text as well.

    # Remove Stopwords
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    sentence = pattern.sub('', sentence)

    return sentence

In [ ]:
df['review'] = df['review'].apply(preprocess_text)
df['review']

0        one reviewers mentioned watching oz episode ho...
1        wonderful little production filming technique ...
2        thought wonderful way spend time hot summer we...
3        basically family little boy jake thinks zombie...
4        petter mattei love time money visually stunnin...
                               ...                        
49995    thought movie right good job creative original...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    one expects star trek movies high art fans exp...
Name: review, Length: 50000, dtype: object

In [ ]:
df['review'][3]

'basically family little boy jake thinks zombie closet parents fighting time movie slower soap opera suddenly jake decides become rambo kill zombie ok first going make film must decide thriller drama drama movie watchable parents divorcing arguing like real life jake closet totally ruins film expected see boogeyman similar movie instead watched drama meaningless thriller spots well playing parents descent dialogs shots jake ignore '

In [ ]:
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x == 'positive'  else 0)
X = df['review']
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=42)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Pad sequences to ensure uniform length
max_len = max([len(seq) for seq in X_train])
X_train = pad_sequences(X_train, padding = 'post',maxlen=max_len)
X_test = pad_sequences(X_test, padding = 'post', maxlen=max_len)

In [ ]:
embedding_path = 'Copy of a2_glove.6B.100d.txt'
embedding_dim = 100

embeddings_index = {}
with open(embedding_path, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < num_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(90703, 100)

In [ ]:
model = Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))

model.add(LSTM(170))
model.add(Dense(1, activation = 'tanh'))

In [ ]:
model.compile(optimizer='adam' , loss= 'binary_crossentropy' , metrics =['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1407, 100)         9070300   
                                                                 
 lstm (LSTM)                 (None, 170)               184280    
                                                                 
 dense (Dense)               (None, 1)                 171       
                                                                 
Total params: 9254751 (35.30 MB)
Trainable params: 184451 (720.51 KB)
Non-trainable params: 9070300 (34.60 MB)
_________________________________________________________________
None


In [ ]:
LSTM = model.fit(X_train,y_train, epochs=7 , batch_size = 120 , validation_split = 0.1)

Epoch 1/7
110/110 [==============================] - 1608s 15s/step - loss: 7.6893 - acc: 0.5015 - val_loss: 7.7703 - val_acc: 0.4963
Epoch 2/7
110/110 [==============================] - 1558s 14s/step - loss: 7.6893 - acc: 0.5015 - val_loss: 7.7703 - val_acc: 0.4963
Epoch 3/7
110/110 [==============================] - 1563s 14s/step - loss: 7.6893 - acc: 0.5015 - val_loss: 7.7703 - val_acc: 0.4963
Epoch 4/7
110/110 [==============================] - 1572s 14s/step - loss: 7.6893 - acc: 0.5015 - val_loss: 7.7703 - val_acc: 0.4963
Epoch 5/7
110/110 [==============================] - 1566s 14s/step - loss: 7.6893 - acc: 0.5015 - val_loss: 7.7703 - val_acc: 0.4963
Epoch 6/7
110/110 [==============================] - 1569s 14s/step - loss: 7.6893 - acc: 0.5015 - val_loss: 7.7703 - val_acc: 0.4963
Epoch 7/7
110/110 [==============================] - 1576s 14s/step - loss: 7.6893 - acc: 0.5015 - val_loss: 7.7703 - val_acc: 0.4963


In [ ]:
evaluation = model.evaluate(X_test,y_test)